## ⚠️ BEFORE YOU START

**What day is it today?**

- ✅ **Friday after 4pm ET:** Perfect! Run all cells below
- ❌ **Monday-Thursday:** STOP! Wait until Friday after market close
- ❌ **Friday before 4pm:** Wait until market closes at 4pm ET

**Why Friday close?** GHB Strategy uses weekly closing prices. Trading mid-week gives false signals and won't match backtested performance.

---

# GHB Strategy Portfolio Scanner
**Gold-Gray-Blue Weekly Trading System**

## 📅 WEEKLY TRADING SCHEDULE

⚠️ **IMPORTANT:** Only run this scanner on **FRIDAY after 4pm ET** (after market close)

### Your Weekly Routine:

**FRIDAY (After 4pm ET)**
- 📊 Run this notebook (all cells)
- 📋 Review signals: BUY (P1), HOLD (P2/N1), SELL (N2)
- 📝 Make your trade list for Monday
- ⏱️ Time: 10-15 minutes

**WEEKEND (Saturday/Sunday)**
- 💭 Review and confirm your plan
- 🧮 Calculate position sizes (10% each = $11,000 per position)
- ✅ Prepare for Monday execution

**MONDAY (Market Open - 9:30am ET)**
- 🔵 **FIRST (9:30-10:00am):** Execute ALL sell signals (N2 stocks) - URGENT
- 🟡 **THEN (10:00-10:30am):** Enter new buy positions (P1 stocks) - PATIENT
- ⏱️ Time: 15-30 minutes
- 💡 Tip: Use limit orders (Sells: Friday -1%, Buys: Friday +1.5%)

---

**Last Run:** {current_date}  
**Strategy:** GHB Strategy (Gold-Gray-Blue)  
**Universe:** 11 AI/Tech Stocks (Focused on AI dominance thesis 2023-2032)  
**Configuration:** 10% position size, 10 max holdings  
**Backtest Performance:** 56.51% CAGR | 40% Win Rate | Best: NVDA +516% (2022-2025)

In [1]:
# Import Required Libraries
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries loaded successfully")
print(f"📅 Current Date: {datetime.now().strftime('%Y-%m-%d')}")

✅ Libraries loaded successfully
📅 Current Date: 2026-01-16


## 1. Define Stock Universe

**AI/Tech Focused Portfolio (12 stocks)**
- **Thesis:** AI will dominate 2023-2032
- **Backtest:** 56.51% CAGR (2022-2025) vs 15.28% for diversified S&P 500
- **Key Winner:** NVDA +516% in single trade (777 days)
- **Concentration Risk:** High - all tech/AI exposure, but aligned with macro thesis

This portfolio captured the 2023-2025 AI boom and is positioned for continued AI infrastructure growth.

In [2]:
# GHB Strategy AI/Tech Focused Portfolio - 13 Stocks
# Selected for AI dominance thesis (2023-2032)
# Backtest (2022-2025): 56.51% CAGR, 332% total return, 40% win rate
GHB_UNIVERSE = [
    'ALAB',  # Astera Labs - AI connectivity
    'AMD',   # Advanced Micro Devices - AI chips
    'ARM',   # ARM Holdings - Mobile AI
    'ASML',  # ASML - Chip equipment
    'AVGO',  # Broadcom - AI infrastructure
    'GOOG',  # Google - AI/Cloud
    'MRVL',  # Marvell - Data infrastructure
    'MU',    # Micron - Memory/AI chips
    'NVDA',  # NVIDIA - AI infrastructure (Best: +516%)
    'PLTR',  # Palantir - AI software
    'TSLA',  # Tesla - AI/Autonomy
    'TSM',   # Taiwan Semi - AI chip manufacturing
]

print(f"📊 Universe: {len(GHB_UNIVERSE)} stocks (AI/Tech Focused)")
print(f"📈 Stocks: {', '.join(sorted(GHB_UNIVERSE[:6]))}...")
print(f"💡 Backtest (2022-2025): 56.51% CAGR | Best: NVDA +516% | Thesis: AI dominance 2023-2032")

📊 Universe: 12 stocks (AI/Tech Focused)
📈 Stocks: ALAB, AMD, ARM, ASML, AVGO, GOOG...
💡 Backtest (2022-2025): 56.51% CAGR | Best: NVDA +516% | Thesis: AI dominance 2023-2032


## 2. Calculate Weekly Larsson States

For each stock, calculate:
- **Weekly Close** (Friday)
- **200-Day SMA** (D200)
- **4-Week ROC** (Rate of Change)
- **RSI (14-period)** (Overextension indicator)
- **Weekly State** (P1/P2/N1/N2)
- **Entry Quality** (Pullback Buy / Healthy / Extended / Overheated)

In [3]:
def calculate_weekly_larsson_state(ticker):
    """
    Calculate weekly Larsson state for a ticker with support/resistance levels
    Returns: dict with ticker data or None if error
    """
    try:
        # Download 1 year of daily data
        stock = yf.Ticker(ticker)
        df = stock.history(period='1y', interval='1d')
        
        if df.empty or len(df) < 200:
            return None
        
        # Get latest close
        close = df['Close'].iloc[-1]
        
        # Calculate 200-day SMA
        d200 = df['Close'].rolling(window=200).mean().iloc[-1]
        
        # Calculate 4-week ROC (20 trading days)
        if len(df) >= 20:
            price_4w_ago = df['Close'].iloc[-20]
            roc_4w = ((close - price_4w_ago) / price_4w_ago) * 100
        else:
            roc_4w = 0
        
        # Calculate distance from D200
        distance_pct = ((close - d200) / d200) * 100
        
        # Calculate RSI (14-period)
        delta = df['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        rs = gain / loss
        rsi = 100 - (100 / (1 + rs))
        rsi_value = rsi.iloc[-1]
        
        # === NEW: Calculate Support/Resistance Levels ===
        
        # 52-week high/low
        week_52_high = df['High'].tail(252).max()
        week_52_low = df['Low'].tail(252).min()
        
        # Recent swing low (support) - lowest low in last 60 days
        recent_support = df['Low'].tail(60).min()
        
        # Recent swing high (resistance) - highest high in last 60 days
        recent_resistance = df['High'].tail(60).max()
        
        # 50-day and 100-day SMAs (additional support/resistance)
        sma_50 = df['Close'].rolling(window=50).mean().iloc[-1]
        sma_100 = df['Close'].rolling(window=100).mean().iloc[-1]
        
        # Calculate distance to key levels
        to_support = ((close - recent_support) / close) * 100
        to_resistance = ((recent_resistance - close) / close) * 100
        
        # Risk assessment
        if to_support < 5:
            risk_level = "LOW (Near support)"
        elif to_support < 10:
            risk_level = "MODERATE"
        else:
            risk_level = "HIGH (Far from support)"
        
        # Determine Weekly Larsson state (Strategy D rules)
        if close > d200:
            # Price above D200
            if roc_4w > 5 or distance_pct > 10:
                state = 'P1'  # Strong bullish
                signal = '🟡 BUY'
            else:
                state = 'P2'  # Consolidation
                signal = '⚪ HOLD'
        else:
            # Price below D200
            if distance_pct > -5:
                state = 'N1'  # Shallow pullback
                signal = '⚪ HOLD'
            else:
                state = 'N2'  # Downtrend
                signal = '🔵 SELL'
        
        return {
            'Ticker': ticker,
            'Close': close,
            'D200': d200,
            'Distance_%': distance_pct,
            'ROC_4W_%': roc_4w,
            'RSI': rsi_value,
            'State': state,
            'Signal': signal,
            # Support/Resistance data
            'Support': recent_support,
            'Resistance': recent_resistance,
            'To_Support_%': to_support,
            'To_Resistance_%': to_resistance,
            '52W_High': week_52_high,
            '52W_Low': week_52_low,
            'SMA_50': sma_50,
            'SMA_100': sma_100,
            'Risk_Level': risk_level
        }
        
    except Exception as e:
        print(f"❌ Error processing {ticker}: {str(e)}")
        return None

print("✅ Calculation function defined (with support/resistance levels)")

✅ Calculation function defined (with support/resistance levels)


## 3. Scan All Stocks

This will take ~30 seconds to download data and calculate states...

In [4]:
print(f"🔄 Scanning {len(GHB_UNIVERSE)} stocks... Please wait...\n")

results = []
for i, ticker in enumerate(GHB_UNIVERSE, 1):
    print(f"  [{i:2d}/{len(GHB_UNIVERSE)}] Processing {ticker:6s}...", end='\r')
    result = calculate_weekly_larsson_state(ticker)
    if result:
        results.append(result)

df_results = pd.DataFrame(results)

print(f"\n✅ Scan complete! Processed {len(df_results)}/{len(GHB_UNIVERSE)} stocks")
print(f"❌ Failed: {len(GHB_UNIVERSE) - len(df_results)} stocks")

🔄 Scanning 12 stocks... Please wait...

  [12/12] Processing TSM   ...
✅ Scan complete! Processed 12/12 stocks
❌ Failed: 0 stocks


## 4. Strategy D Signals

### Buy Signals (P1 - Gold)
**Action:** Enter new positions or add to existing  
**Requirements:** Price > D200 + Strong momentum

In [5]:
# Filter P1 (BUY) signals
p1_signals = df_results[df_results['State'] == 'P1'].copy()

# Add overextension categorization
def categorize_entry(row):
    rsi = row['RSI']
    distance = row['Distance_%']
    roc = row['ROC_4W_%']
    
    # Priority 1: Pullback buy (P1 but negative ROC = dip buying)
    if roc < 0:
        return '🔥 PULLBACK BUY', 1
    # Priority 2: Healthy buy (not overextended)
    elif rsi < 70 and distance < 30:
        return '✅ HEALTHY BUY', 2
    # Priority 3: Extended (caution, but tradable)
    elif rsi < 80 or distance < 40:
        return '⚠️ EXTENDED', 3
    # Priority 4: Overheated (high risk)
    else:
        return '🚨 OVERHEATED', 4

p1_signals[['Entry_Quality', 'Priority']] = p1_signals.apply(categorize_entry, axis=1, result_type='expand')

# Create PDF-friendly version without emojis for export
def make_pdf_friendly(quality_text):
    """Remove emojis from quality labels for PDF compatibility"""
    return quality_text.replace('🔥 ', '').replace('✅ ', '').replace('⚠️ ', '').replace('🚨 ', '')

p1_signals['Entry_Quality_PDF'] = p1_signals['Entry_Quality'].apply(make_pdf_friendly)

# Sort by Priority (entry quality) first, then by distance from support (safest entries first)
p1_signals = p1_signals.sort_values(['Priority', 'To_Support_%'])

print("=" * 120)
print(f"🟡 P1 (GOLD) - BUY SIGNALS: {len(p1_signals)} stocks")
print("=" * 120)

if len(p1_signals) > 0:
    print(f"\n{'Ticker':<8} {'Price':<10} {'D200':<10} {'Dist %':<10} {'ROC %':<10} {'RSI':<6} {'Entry Quality':<20}")
    print("-" * 120)
    
    for _, row in p1_signals.iterrows():
        print(f"{row['Ticker']:<8} ${row['Close']:<9.2f} ${row['D200']:<9.2f} {row['Distance_%']:>+8.1f}% {row['ROC_4W_%']:>+8.1f}% {row['RSI']:>5.0f} {row['Entry_Quality']:<20}")
    
    # Count by category
    pullback = len(p1_signals[p1_signals['Entry_Quality'] == '🔥 PULLBACK BUY'])
    healthy = len(p1_signals[p1_signals['Entry_Quality'] == '✅ HEALTHY BUY'])
    extended = len(p1_signals[p1_signals['Entry_Quality'] == '⚠️ EXTENDED'])
    overheated = len(p1_signals[p1_signals['Entry_Quality'] == '🚨 OVERHEATED'])
    
    print("\n📊 ENTRY QUALITY BREAKDOWN:")
    if pullback > 0:
        print(f"   🔥 PULLBACK BUY: {pullback} stock(s) - PRIORITY #1 (P1 state but dipping)")
    if healthy > 0:
        print(f"   ✅ HEALTHY BUY: {healthy} stock(s) - Good entry points")
    if extended > 0:
        print(f"   ⚠️ EXTENDED: {extended} stock(s) - Enter cautiously or wait for dip")
    if overheated > 0:
        print(f"   🚨 OVERHEATED: {overheated} stock(s) - High risk, consider waiting")
    
    print("\n💡 SMART ENTRY STRATEGY:")
    print("   1. FIRST: Enter Pullback Buys (P1 + negative ROC = dip buying opportunity)")
    print("   2. SECOND: Enter Healthy Buys (RSI <70, not overextended)")
    print("   3. CAUTION: Extended stocks may pull back - enter small or wait")
    print("   4. AVOID: Overheated stocks (RSI >80 + Distance >40%) - wait for consolidation")
else:
    print("\n⚠️  No P1 buy signals this week")
    print("   - Wait for new P1 entries")
    print("   - Hold existing positions")
    print("   - Consider if market environment has changed")

🟡 P1 (GOLD) - BUY SIGNALS: 10 stocks

Ticker   Price      D200       Dist %     ROC %      RSI    Entry Quality       
------------------------------------------------------------------------------------------------------------------------
TSLA     $438.57    $368.28       +19.1%     -6.1%    29 🔥 PULLBACK BUY      
PLTR     $177.07    $155.01       +14.2%     -0.1%    33 🔥 PULLBACK BUY      
AVGO     $343.02    $296.24       +15.8%     +5.4%    45 ✅ HEALTHY BUY       
NVDA     $187.05    $163.66       +14.3%     +9.4%    47 ✅ HEALTHY BUY       
ALAB     $174.45    $139.11       +25.4%    +24.4%    52 ✅ HEALTHY BUY       
AMD      $227.92    $168.05       +35.6%    +15.0%    61 ⚠️ EXTENDED         
TSM      $341.64    $244.78       +39.6%    +23.4%    78 ⚠️ EXTENDED         
GOOG     $333.16    $226.22       +47.3%    +11.8%    76 ⚠️ EXTENDED         
MU       $336.63    $159.88      +110.6%    +49.3%    68 ⚠️ EXTENDED         
ASML     $1331.60   $868.27       +53.4%    +31.1%    82 🚨

### Support/Resistance Levels for P1 Stocks
**Technical analysis for scaled entry decisions**

In [6]:
# Display support/resistance analysis for P1 signals
if len(p1_signals) > 0:
    print("\n" + "=" * 140)
    print("📍 SUPPORT/RESISTANCE ANALYSIS - P1 BUY SIGNALS")
    print("=" * 140)
    print(f"\n{'Ticker':<8} {'Current':<10} {'Support':<10} {'To Supp':<10} {'Resistance':<12} {'To Resist':<11} {'Risk Level':<20}")
    print("-" * 140)
    
    for _, row in p1_signals.iterrows():
        print(f"{row['Ticker']:<8} ${row['Close']:<9.2f} ${row['Support']:<9.2f} "
              f"{row['To_Support_%']:>+8.1f}% ${row['Resistance']:<11.2f} "
              f"{row['To_Resistance_%']:>+8.1f}% {row['Risk_Level']:<20}")
    
    print("\n📊 KEY LEVELS SUMMARY:")
    print(f"{'Ticker':<8} {'52W Low':<12} {'SMA 50':<12} {'SMA 100':<12} {'D200':<12} {'52W High':<12}")
    print("-" * 140)
    
    for _, row in p1_signals.iterrows():
        print(f"{row['Ticker']:<8} ${row['52W_Low']:<11.2f} ${row['SMA_50']:<11.2f} "
              f"${row['SMA_100']:<11.2f} ${row['D200']:<11.2f} ${row['52W_High']:<11.2f}")
    
    # Sort by distance from support first (safer entries prioritized), then by momentum quality
    p1_signals = p1_signals.sort_values(['To_Support_%', 'Priority'])
    
    print("\n💡 SCALED ENTRY RECOMMENDATIONS:")
    print("-" * 140)
    
    for _, row in p1_signals.iterrows():
        ticker = row['Ticker']
        current = row['Close']
        support = row['Support']
        to_support = row['To_Support_%']
        risk = row['Risk_Level']
        
        # Calculate entry strategy based on distance from support
        if to_support < 3:
            # Very close to support - safe full entry
            rec = f"✅ FULL POSITION - Near support ${support:.2f}, low downside risk"
            size_pct = 100
        elif to_support < 5:
            # Close to support - 75% position
            rec = f"🟢 75% POSITION - {to_support:.1f}% above support, good entry point"
            size_pct = 75
        elif to_support < 10:
            # Moderate distance - 50% pilot
            rec = f"🟡 50% PILOT - {to_support:.1f}% above support, consider scaling"
            size_pct = 50
        else:
            # Far from support - 25-40% pilot or wait
            rec = f"⚠️  25-40% PILOT - {to_support:.1f}% above support, high risk or WAIT for dip to ${support:.2f}"
            size_pct = 30
        
        print(f"\n{ticker}: {rec}")
        print(f"   Entry Plan: {size_pct}% now @ ${current:.2f}, add on dip to ${support*1.02:.2f}-${support*1.05:.2f}")
        
    print("\n" + "=" * 140)
else:
    print("\n⚠️  No P1 signals - No support analysis available")


📍 SUPPORT/RESISTANCE ANALYSIS - P1 BUY SIGNALS

Ticker   Current    Support    To Supp    Resistance   To Resist   Risk Level          
--------------------------------------------------------------------------------------------------------------------------------------------
TSLA     $438.57    $382.78       +12.7% $498.83         +13.7% HIGH (Far from support)
PLTR     $177.07    $147.56       +16.7% $207.52         +17.2% HIGH (Far from support)
AVGO     $343.02    $320.81        +6.5% $413.82         +20.6% MODERATE            
NVDA     $187.05    $169.54        +9.4% $212.18         +13.4% MODERATE            
ALAB     $174.45    $131.42       +24.7% $201.86         +15.7% HIGH (Far from support)
AMD      $227.92    $194.28       +14.8% $267.08         +17.2% HIGH (Far from support)
TSM      $341.64    $266.10       +22.1% $351.33          +2.8% HIGH (Far from support)
GOOG     $333.16    $244.51       +26.6% $341.20          +2.4% HIGH (Far from support)
MU       $336.63    $192

### Hold Signals (P2 & N1 - Gray)
**Action:** Continue holding existing positions  
**Meaning:** Normal consolidation/pullback in trend

In [7]:
# Filter P2 and N1 (HOLD) signals
hold_signals = df_results[df_results['State'].isin(['P2', 'N1'])].sort_values('Distance_%', ascending=False)

print("=" * 100)
print(f"⚪ P2/N1 (GRAY) - HOLD SIGNALS: {len(hold_signals)} stocks")
print("=" * 100)

if len(hold_signals) > 0:
    print(f"\n{'Ticker':<8} {'Price':<10} {'D200':<10} {'Distance':<12} {'4W ROC':<10} {'State':<8} {'Status':<20}")
    print("-" * 100)
    
    for _, row in hold_signals.iterrows():
        if row['State'] == 'P2':
            status = "Consolidation"
        else:
            status = "Shallow Pullback"
        
        print(f"{row['Ticker']:<8} ${row['Close']:<9.2f} ${row['D200']:<9.2f} {row['Distance_%']:>+10.1f}% {row['ROC_4W_%']:>+8.2f}% {row['State']:<8} {status:<20}")
    
    print("\n💡 RECOMMENDATION:")
    print("   - HOLD all existing positions")
    print("   - Do NOT sell - this is normal consolidation")
    print("   - Watch for transition to P1 (upgrade) or N2 (downgrade)")
else:
    print("\n✅ No stocks in consolidation phase")

⚪ P2/N1 (GRAY) - HOLD SIGNALS: 1 stocks

Ticker   Price      D200       Distance     4W ROC     State    Status              
----------------------------------------------------------------------------------------------------
MRVL     $80.38     $75.04           +7.1%    -1.54% P2       Consolidation       

💡 RECOMMENDATION:
   - HOLD all existing positions
   - Do NOT sell - this is normal consolidation
   - Watch for transition to P1 (upgrade) or N2 (downgrade)


### Sell Signals (N2 - Blue)
**Action:** Exit positions immediately  
**Requirements:** Price < D200 + Weak momentum  
**⚠️ CRITICAL: Execute these sells on Monday!**

In [8]:
# Filter N2 (SELL) signals
n2_signals = df_results[df_results['State'] == 'N2'].sort_values('Distance_%', ascending=True)

print("=" * 100)
print(f"🔵 N2 (BLUE) - SELL SIGNALS: {len(n2_signals)} stocks")
print("=" * 100)

if len(n2_signals) > 0:
    print(f"\n⚠️  EXIT THESE POSITIONS ON MONDAY!\n")
    print(f"{'Ticker':<8} {'Price':<10} {'D200':<10} {'Below D200':<12} {'4W ROC':<10} {'Severity':<15}")
    print("-" * 100)
    
    for _, row in n2_signals.iterrows():
        # Determine severity
        if row['Distance_%'] < -20:
            severity = "🚨 SEVERE"
        elif row['Distance_%'] < -10:
            severity = "⚠️  MAJOR"
        else:
            severity = "📉 MINOR"
        
        print(f"{row['Ticker']:<8} ${row['Close']:<9.2f} ${row['D200']:<9.2f} {row['Distance_%']:>+10.1f}% {row['ROC_4W_%']:>+8.2f}% {severity:<15}")
    
    print("\n💡 ACTION REQUIRED:")
    print("   - SELL all N2 positions on Monday at market open")
    print("   - Do NOT wait for bounce - trend is broken")
    print("   - Preserve capital for new P1 opportunities")
else:
    print("\n✅ No sell signals - all positions healthy!")

🔵 N2 (BLUE) - SELL SIGNALS: 1 stocks

⚠️  EXIT THESE POSITIONS ON MONDAY!

Ticker   Price      D200       Below D200   4W ROC     Severity       
----------------------------------------------------------------------------------------------------
ARM      $105.11    $137.19         -23.4%    -8.26% 🚨 SEVERE       

💡 ACTION REQUIRED:
   - SELL all N2 positions on Monday at market open
   - Do NOT wait for bounce - trend is broken
   - Preserve capital for new P1 opportunities


## 5. Weekly Summary

Quick overview of portfolio status and action items

In [9]:
print("=" * 100)
print("📊 GHB STRATEGY WEEKLY SUMMARY")
print("=" * 100)

print(f"\n🟡 BUY Signals (P1):  {len(p1_signals)} stocks")
print(f"⚪ HOLD Signals (P2/N1): {len(hold_signals)} stocks")
print(f"🔵 SELL Signals (N2): {len(n2_signals)} stocks")
print(f"📊 Total Scanned: {len(df_results)}/{len(GHB_UNIVERSE)} stocks")

# Calculate portfolio health
total_bullish = len(p1_signals)
total_neutral = len(hold_signals)
total_bearish = len(n2_signals)
total = len(df_results)

pct_bullish = (total_bullish / total * 100) if total > 0 else 0
pct_bearish = (total_bearish / total * 100) if total > 0 else 0

print(f"\n📈 Market Health:")
print(f"   Bullish: {pct_bullish:.1f}% ({total_bullish} stocks)")
print(f"   Neutral: {(total_neutral/total*100):.1f}% ({total_neutral} stocks)")
print(f"   Bearish: {pct_bearish:.1f}% ({total_bearish} stocks)")

if pct_bullish > 60:
    market_sentiment = "🟢 VERY BULLISH - Many opportunities"
elif pct_bullish > 40:
    market_sentiment = "🟡 BULLISH - Good opportunities"
elif pct_bullish > 20:
    market_sentiment = "🟠 NEUTRAL - Selective opportunities"
else:
    market_sentiment = "🔴 BEARISH - Few opportunities, preserve cash"

print(f"\n📊 Market Sentiment: {market_sentiment}")

# Action items
print("\n✅ ACTION ITEMS FOR THIS WEEK:")
if len(n2_signals) > 0:
    print(f"   1. MONDAY: Sell {len(n2_signals)} N2 positions at market open")
else:
    print("   1. No sells required")

if len(p1_signals) > 0:
    print(f"   2. MONDAY: Enter up to {min(5, len(p1_signals))} new P1 positions")
    print(f"      → Priority: {', '.join(p1_signals.head(5)['Ticker'].tolist())}")
else:
    print("   2. No new buys available - hold cash")

if len(hold_signals) > 0:
    print(f"   3. Monitor {len(hold_signals)} holding positions for state changes")
else:
    print("   3. No positions to monitor")

print("\n" + "=" * 100)

📊 GHB STRATEGY WEEKLY SUMMARY

🟡 BUY Signals (P1):  10 stocks
⚪ HOLD Signals (P2/N1): 1 stocks
🔵 SELL Signals (N2): 1 stocks
📊 Total Scanned: 12/12 stocks

📈 Market Health:
   Bullish: 83.3% (10 stocks)
   Neutral: 8.3% (1 stocks)
   Bearish: 8.3% (1 stocks)

📊 Market Sentiment: 🟢 VERY BULLISH - Many opportunities

✅ ACTION ITEMS FOR THIS WEEK:
   1. MONDAY: Sell 1 N2 positions at market open
   2. MONDAY: Enter up to 5 new P1 positions
      → Priority: AVGO, NVDA, TSLA, AMD, PLTR
   3. Monitor 1 holding positions for state changes



## 6. Detailed Stock Data

Full dataset for analysis and record-keeping

In [10]:
# Display full results sorted by state then distance
df_display = df_results.copy()
df_display['State_Order'] = df_display['State'].map({'P1': 1, 'P2': 2, 'N1': 3, 'N2': 4})
df_display = df_display.sort_values(['State_Order', 'Distance_%'], ascending=[True, False])
df_display = df_display.drop('State_Order', axis=1)

print("\n📋 COMPLETE SCAN RESULTS")
print("=" * 100)
print(df_display.to_string(index=False))
print("=" * 100)


📋 COMPLETE SCAN RESULTS
Ticker       Close       D200  Distance_%  ROC_4W_%       RSI State Signal    Support  Resistance  To_Support_%  To_Resistance_%    52W_High    52W_Low      SMA_50     SMA_100              Risk_Level
    MU  336.630005 159.877043  110.555560 49.328655 68.384333    P1  🟡 BUY 192.322306  351.230011     42.868341         4.337108  351.230011  61.418842  263.111514  217.566581 HIGH (Far from support)
  ASML 1331.599976 868.267527   53.362867 31.136561 82.065551    P1  🟡 BUY 946.109985 1358.000000     28.949384         1.982579 1358.000000 574.245917 1093.281588 1011.466008 HIGH (Far from support)
  GOOG  333.160004 226.224132   47.269878 11.776155 76.003550    P1  🟡 BUY 244.510468  341.200012     26.608697         2.413257  341.200012 142.268661  308.916461  277.362521 HIGH (Far from support)
   TSM  341.640015 244.778884   39.570869 23.353562 77.631589    P1  🟡 BUY 266.101623  351.329987     22.110522         2.836311  351.329987 132.981258  297.817054  286.274012

## 6.5 Current Portfolio Holdings

**Phase 1 - Portfolio Tracker:** Track your positions and calculate performance

In [11]:
import json
from pathlib import Path

# Load portfolio settings
settings_path = Path("../data/portfolio_settings.json")
with open(settings_path, 'r') as f:
    portfolio_settings = json.load(f)

starting_cash = portfolio_settings['starting_cash']
position_size_pct = portfolio_settings['position_size_pct']
max_positions = portfolio_settings['max_positions']
strategy_week = portfolio_settings['strategy_week']
conservative_mode = portfolio_settings.get('conservative_mode', True)
position_allocations = portfolio_settings.get('position_allocations', {})

# Helper function to get position size for a ticker
def get_position_allocation(ticker):
    """Get allocation percentage for a ticker (custom or default)"""
    # If ticker has custom allocation, return it
    if ticker in position_allocations:
        return position_allocations[ticker]
    
    # Calculate remaining allocation for non-custom stocks
    total_custom_alloc = sum(position_allocations.values())
    remaining_alloc = 100 - total_custom_alloc
    
    # Split remaining allocation among (max_positions - custom_count) stocks
    num_non_custom_slots = max_positions - len(position_allocations)
    if num_non_custom_slots > 0:
        return remaining_alloc / num_non_custom_slots
    else:
        return position_size_pct  # Fallback if all positions are custom

def get_position_value(ticker):
    """Get dollar value for a position based on allocation"""
    return starting_cash * get_position_allocation(ticker) / 100

print("💼 PORTFOLIO CONFIGURATION")
print("=" * 100)
print(f"Starting Capital: ${starting_cash:,.0f}")
print(f"Default Position Size: {position_size_pct}% (${starting_cash * position_size_pct / 100:,.0f} per position)")
print(f"Max Positions: {max_positions}")
print(f"Strategy Week: {strategy_week}")
print(f"Mode: {'Conservative (Building Gradually)' if conservative_mode else 'Aggressive'}")
if position_allocations:
    print(f"\n⚖️  Custom Allocations:")
    for ticker, pct in sorted(position_allocations.items(), key=lambda x: x[1], reverse=True):
        print(f"   {ticker}: {pct}% (${starting_cash * pct / 100:,.0f})")
print("=" * 100)

# Load current positions
positions_path = Path("../data/portfolio_positions.csv")
df_positions = pd.read_csv(positions_path)

# Enrich with current signals for PDF generation
df_positions_enriched = df_positions.copy()
if len(df_positions) > 0:
    signals_list = []
    prices_list = []
    pl_pct_list = []
    for _, pos in df_positions.iterrows():
        ticker = pos['Ticker']
        current_data = df_results[df_results['Ticker'] == ticker]
        if len(current_data) > 0:
            signals_list.append(current_data.iloc[0]['Signal'])
            current_price = current_data.iloc[0]['Close']
            prices_list.append(current_price)
            pl_pct = ((current_price - pos['Entry_Price']) / pos['Entry_Price']) * 100
            pl_pct_list.append(pl_pct)
        else:
            signals_list.append('UNKNOWN')
            prices_list.append(pos['Entry_Price'])
            pl_pct_list.append(0)
    
    df_positions_enriched['Signal'] = signals_list
    df_positions_enriched['Current_Price'] = prices_list
    df_positions_enriched['P/L_%'] = pl_pct_list
    df_positions_enriched['Position_Value'] = df_positions_enriched['Shares'] * df_positions_enriched['Current_Price']
    df_positions_enriched['P/L_$'] = (df_positions_enriched['Current_Price'] - df_positions_enriched['Entry_Price']) * df_positions_enriched['Shares']

# Initialize variables for both cases
position_summaries = []
total_cost = 0
total_value = 0
total_pl = 0
total_pl_pct = 0
cash_remaining = starting_cash
deployed_pct = 0

if len(df_positions) == 0:
    print("\n📭 No positions yet - Portfolio is 100% CASH")
    print(f"💰 Available: ${starting_cash:,.0f}")
    print(f"\n💡 Week {strategy_week} Recommendation:")
    if strategy_week == 1:
        print("   Start with 2-3 positions")
        # Calculate suggested allocation based on available P1 signals with custom allocations
        if len(p1_signals) > 0:
            top_picks = p1_signals.head(3)['Ticker'].tolist()
            suggested_deploy = sum([get_position_allocation(t) for t in top_picks])
            print(f"   Suggested: {', '.join(top_picks)}")
            print(f"   Total Deploy: {suggested_deploy:.1f}% (${starting_cash * suggested_deploy / 100:,.0f})")
            for ticker in top_picks:
                alloc = get_position_allocation(ticker)
                print(f"      {ticker}: {alloc}% = ${starting_cash * alloc / 100:,.0f}")
    else:
        print(f"   Consider adding 2-3 new positions")
        print(f"   Build towards {max_positions} total positions")
else:
    print(f"\n📊 ACTIVE POSITIONS: {len(df_positions)}")
    print("=" * 100)
    
    # Update current states for all positions
    position_summaries = []
    
    for _, pos in df_positions.iterrows():
        ticker = pos['Ticker']
        
        # Get current data from scan results
        current_data = df_results[df_results['Ticker'] == ticker]
        
        if len(current_data) > 0:
            current_price = current_data.iloc[0]['Close']
            current_state = current_data.iloc[0]['State']
            current_signal = current_data.iloc[0]['Signal']
            distance_pct = current_data.iloc[0]['Distance_%']
            roc_pct = current_data.iloc[0]['ROC_4W_%']
        else:
            # Ticker not in universe or failed to scan
            current_price = pos['Entry_Price']  # Fallback
            current_state = pos['Current_State']
            current_signal = '❓ UNKNOWN'
            distance_pct = 0
            roc_pct = 0
        
        # Calculate P&L
        entry_price = pos['Entry_Price']
        shares = pos['Shares']
        cost_basis = entry_price * shares
        current_value = current_price * shares
        pl_dollars = current_value - cost_basis
        pl_pct = ((current_price - entry_price) / entry_price) * 100
        
        # Get allocation for display
        target_alloc = get_position_allocation(ticker)
        actual_alloc = (cost_basis / starting_cash) * 100
        
        # State change detection
        state_change = ""
        if current_state != pos['Entry_State']:
            state_change = f"({pos['Entry_State']} → {current_state})"
        
        position_summaries.append({
            'Ticker': ticker,
            'Allocation': f"{actual_alloc:.1f}%",
            'Entry_Date': pos['Entry_Date'],
            'Entry_Price': f"${entry_price:.2f}",
            'Current_Price': f"${current_price:.2f}",
            'Shares': int(shares),
            'Cost_Basis': f"${cost_basis:,.0f}",
            'Current_Value': f"${current_value:,.0f}",
            'P/L_$': f"${pl_dollars:+,.0f}",
            'P/L_%': f"{pl_pct:+.1f}%",
            'Entry_State': pos['Entry_State'],
            'Current_State': current_state,
            'State_Change': state_change,
            'Signal': current_signal
        })
    
    df_summary = pd.DataFrame(position_summaries)
    print(df_summary.to_string(index=False))
    print("=" * 100)
    
    # Portfolio totals
    total_cost = sum([float(s['Cost_Basis'].replace('$', '').replace(',', '')) for s in position_summaries])
    total_value = sum([float(s['Current_Value'].replace('$', '').replace(',', '')) for s in position_summaries])
    total_pl = total_value - total_cost
    total_pl_pct = (total_pl / total_cost * 100) if total_cost > 0 else 0
    
    cash_remaining = starting_cash - total_cost
    deployed_pct = (total_cost / starting_cash * 100)
    
    print(f"\n💼 PORTFOLIO SUMMARY")
    print("=" * 100)
    print(f"Total Cost Basis: ${total_cost:,.0f}")
    print(f"Current Value: ${total_value:,.0f}")
    print(f"Total P/L: ${total_pl:+,.0f} ({total_pl_pct:+.1f}%)")
    print(f"Cash Remaining: ${cash_remaining:,.0f}")
    print(f"Deployed: {deployed_pct:.1f}% | Cash: {100-deployed_pct:.1f}%")
    print("=" * 100)
    
    # Alerts and Warnings
    print(f"\n⚠️ PORTFOLIO ALERTS")
    print("=" * 100)
    
    n2_positions = [s for s in position_summaries if 'SELL' in s['Signal']]
    state_changes = [s for s in position_summaries if s['State_Change'] != ""]
    
    if len(n2_positions) > 0:
        print(f"🔴 URGENT: {len(n2_positions)} position(s) in N2 (SELL) state!")
        for pos in n2_positions:
            print(f"   → {pos['Ticker']}: SELL on Monday open (Current: {pos['Current_State']})")
    else:
        print("✅ No urgent sell signals")
    
    if len(state_changes) > 0:
        print(f"\n📊 {len(state_changes)} position(s) changed state:")
        for pos in state_changes:
            print(f"   → {pos['Ticker']}: {pos['State_Change']}")
    else:
        print("\n✅ All positions maintained their states")
    
    print("=" * 100)
    
    # Position sizing recommendations with variable allocations
    print(f"\n💰 POSITION SIZING FOR THIS WEEK")
    print("=" * 100)
    
    current_positions = len(df_positions)
    positions_to_add = max_positions - current_positions
    
    if positions_to_add > 0 and len(p1_signals) > 0:
        print(f"📈 Recommended: Add {min(positions_to_add, len(p1_signals))} new position(s)")
        print(f"\n💡 Top {min(positions_to_add, 5)} Candidates with Allocations:")
        
        total_deploy = 0
        for i, (_, row) in enumerate(p1_signals.head(min(positions_to_add, 5)).iterrows()):
            ticker = row['Ticker']
            alloc_pct = get_position_allocation(ticker)
            position_value = get_position_value(ticker)
            total_deploy += position_value
            
            quality = row.get('Entry_Quality', 'N/A')
            price = row['Close']
            
            print(f"   {i+1}. {ticker}: {alloc_pct}% = ${position_value:,.0f} @ ${price:.2f} - {quality}")
        
        print(f"\n🎯 Total to Deploy: ${total_deploy:,.0f}")
        print(f"📊 New Portfolio Allocation: {(total_cost + total_deploy) / starting_cash * 100:.1f}%")
        print(f"💵 Remaining Cash: ${cash_remaining - total_deploy:,.0f}")
        
    elif len(df_positions) >= max_positions:
        print(f"✅ Portfolio full ({len(df_positions)}/{max_positions} positions)")
        print(f"💡 Only trade if N2 sell creates opening")
    else:
        print(f"⚠️  No P1 signals available for new positions")
        print(f"💵 Cash Available: ${cash_remaining:,.0f}")
        print(f"🔍 Wait for next week's scan")

💼 PORTFOLIO CONFIGURATION
Starting Capital: $110,000
Default Position Size: 10% ($11,000 per position)
Max Positions: 10
Strategy Week: 1
Mode: Conservative (Building Gradually)

⚖️  Custom Allocations:
   TSLA: 50% ($55,000)
   NVDA: 20% ($22,000)

📭 No positions yet - Portfolio is 100% CASH
💰 Available: $110,000

💡 Week 1 Recommendation:
   Start with 2-3 positions
   Suggested: AVGO, NVDA, TSLA
   Total Deploy: 73.8% ($81,125)
      AVGO: 3.75% = $4,125
      NVDA: 20% = $22,000
      TSLA: 50% = $55,000


## 6.6 How to Add New Positions

**MANUAL WORKFLOW (Phase 1):**

1. **Friday:** Run this notebook to get signals
2. **Monday:** Execute trades (SELL first, then BUY)
3. **Monday Evening:** Manually update `data/portfolio_positions.csv`

**Example Entry:**
```
TSLA,2026-01-20,450.00,17,P1,P1,🟡 BUY
```

**Fields:**
- `Ticker`: Stock symbol
- `Entry_Date`: YYYY-MM-DD format
- `Entry_Price`: Your fill price
- `Shares`: Number of shares purchased
- `Entry_State`: P1 (the state when you bought)
- `Current_State`: P1 (will auto-update next week)
- `Entry_Signal`: 🟡 BUY

Next Friday, the scanner will automatically update Current_State and calculate P/L!

In [12]:
"""
Universe Health Check & Re-Optimization Alerts
Automatically flag conditions that suggest universe needs refresh
"""

from datetime import datetime, timedelta

print("\n" + "="*80)
print("🔍 UNIVERSE HEALTH CHECK - Re-Optimization Alerts")
print("="*80)

# Initialize alert tracking
reopt_alerts = []
alert_severity = "GREEN"  # GREEN, YELLOW, RED

# --- CONDITION 1: Universe Degradation (>30% in N2) ---
n2_count = len(n2_signals)
total_universe = len(df_results)
n2_percentage = (n2_count / total_universe * 100) if total_universe > 0 else 0

print(f"\n📊 Condition 1: Universe Degradation")
print(f"   N2 Stocks: {n2_count}/{total_universe} ({n2_percentage:.1f}%)")

if n2_percentage > 30:
    severity = "🔴 CRITICAL"
    alert_severity = "RED"
    reopt_alerts.append({
        'condition': 'Universe Degradation',
        'severity': 'CRITICAL',
        'detail': f'{n2_percentage:.1f}% of universe in N2 (threshold: 30%)',
        'action': 'Re-screen S&P 500 immediately - universe is broken'
    })
    print(f"   {severity}: {n2_percentage:.1f}% in N2 (Threshold: 30%)")
    print(f"   ⚠️  ACTION: Re-screen S&P 500 NOW - universe showing widespread weakness")
elif n2_percentage > 20:
    severity = "🟡 WARNING"
    if alert_severity == "GREEN":
        alert_severity = "YELLOW"
    reopt_alerts.append({
        'condition': 'Universe Degradation',
        'severity': 'WARNING',
        'detail': f'{n2_percentage:.1f}% of universe in N2 (watch threshold: 20%)',
        'action': 'Monitor closely - consider re-screening if persists 2+ weeks'
    })
    print(f"   {severity}: {n2_percentage:.1f}% in N2 (Watch at 20%)")
    print(f"   💡 Monitor: If this persists for 2+ weeks, consider re-screening")
else:
    print(f"   ✅ HEALTHY: {n2_percentage:.1f}% in N2 (Normal: <20%)")

# --- CONDITION 2: Performance Tracking (if portfolio exists) ---
print(f"\n📈 Condition 2: Portfolio Performance vs Expected")

if len(df_positions) > 0:
    # Calculate expected return based on time held
    # Expected: 56.51% annual = 1.09% per week
    expected_weekly_return = 1.09  # 56.51% / 52 weeks
    
    # Calculate actual return
    total_pl_pct_calc = ((total_value - total_cost) / total_cost * 100) if total_cost > 0 else 0
    
    # Estimate weeks held (simplified - assumes all positions entered at same time)
    weeks_held = strategy_week  # Using strategy week as proxy
    expected_return = expected_weekly_return * weeks_held
    
    performance_gap = total_pl_pct_calc - expected_return
    
    print(f"   Actual Return: {total_pl_pct_calc:+.1f}%")
    print(f"   Expected Return ({weeks_held} weeks): {expected_return:+.1f}%")
    print(f"   Performance Gap: {performance_gap:+.1f}%")
    
    if performance_gap < -10 and weeks_held > 12:
        severity = "🟡 WARNING"
        if alert_severity == "GREEN":
            alert_severity = "YELLOW"
        reopt_alerts.append({
            'condition': 'Performance Lag',
            'severity': 'WARNING',
            'detail': f'Portfolio underperforming by {abs(performance_gap):.1f}% (threshold: -10%)',
            'action': 'Review stock selection - consider re-screening'
        })
        print(f"   {severity}: Underperforming by {abs(performance_gap):.1f}%")
    else:
        print(f"   ✅ ON TRACK: Performance within expected range")
else:
    print(f"   ⏳ No positions yet - tracking starts after first trades")

# --- CONDITION 3: Low Opportunity Environment ---
print(f"\n🎯 Condition 3: Market Opportunities")

p1_percentage = (len(p1_signals) / total_universe * 100) if total_universe > 0 else 0
print(f"   P1 (BUY) Signals: {len(p1_signals)}/{total_universe} ({p1_percentage:.1f}%)")

if p1_percentage < 20:
    severity = "🟡 WARNING"
    if alert_severity == "GREEN":
        alert_severity = "YELLOW"
    reopt_alerts.append({
        'condition': 'Low Opportunity',
        'severity': 'WARNING',
        'detail': f'Only {p1_percentage:.1f}% showing BUY signals',
        'action': 'Market may be bearish - check if universe needs refresh'
    })
    print(f"   {severity}: Very few opportunities")
else:
    print(f"   ✅ HEALTHY: {p1_percentage:.1f}% in P1")

# --- CONDITION 4: Time Since Last Update ---
print(f"\n📅 Condition 4: Universe Age")

portfolio_file = Path("../data/ghb_optimized_portfolio.txt")
if portfolio_file.exists():
    last_modified = datetime.fromtimestamp(portfolio_file.stat().st_mtime)
    days_since = (datetime.now() - last_modified).days
    months_since = days_since / 30.44
    
    print(f"   Last Updated: {last_modified.strftime('%Y-%m-%d')} ({months_since:.1f} months ago)")
    
    if days_since > 365:
        severity = "🔴 CRITICAL"
        alert_severity = "RED"
        reopt_alerts.append({
            'condition': 'Stale Universe',
            'severity': 'CRITICAL',
            'detail': f'Universe {months_since:.1f} months old',
            'action': 'REQUIRED: Annual re-optimization overdue'
        })
        print(f"   {severity}: Over 1 year old - RE-OPTIMIZE NOW")
    elif days_since > 180:
        severity = "🟡 WARNING"
        if alert_severity == "GREEN":
            alert_severity = "YELLOW"
        print(f"   {severity}: Over 6 months - plan update soon")
    else:
        print(f"   ✅ FRESH: Recently updated")

# --- SUMMARY ---
print(f"\n" + "="*80)
print(f"🚦 OVERALL STATUS: {alert_severity}")
print("="*80)

if alert_severity == "RED":
    print(f"🔴 CRITICAL: Re-optimization REQUIRED ({len(reopt_alerts)} issue(s))")
elif alert_severity == "YELLOW":
    print(f"🟡 WARNING: Re-optimization recommended ({len(reopt_alerts)} warning(s))")
else:
    print(f"✅ HEALTHY: No re-optimization needed")

if len(reopt_alerts) > 0:
    print(f"\n📋 ACTION ITEMS:")
    for i, alert in enumerate(reopt_alerts, 1):
        print(f"   {i}. {alert['condition']}: {alert['action']}")
    print(f"\n🔧 TO RE-OPTIMIZE: Run universe_reoptimization.ipynb")

print(f"\n" + "="*80)


🔍 UNIVERSE HEALTH CHECK - Re-Optimization Alerts

📊 Condition 1: Universe Degradation
   N2 Stocks: 1/12 (8.3%)
   ✅ HEALTHY: 8.3% in N2 (Normal: <20%)

📈 Condition 2: Portfolio Performance vs Expected
   ⏳ No positions yet - tracking starts after first trades

🎯 Condition 3: Market Opportunities
   P1 (BUY) Signals: 10/12 (83.3%)
   ✅ HEALTHY: 83.3% in P1

📅 Condition 4: Universe Age
   Last Updated: 2026-01-15 (0.0 months ago)
   ✅ FRESH: Recently updated

🚦 OVERALL STATUS: GREEN
✅ HEALTHY: No re-optimization needed



## 6.4 Universe Health Check & Re-Optimization Alerts

**Automated monitoring:** Check if universe needs refresh

## 6.7 Monday Execution Guidelines

### ⏰ TIMING STRATEGY

**9:30-10:00am: SELL N2 Positions (URGENT)**
- Execute ALL sells in first 30 minutes
- N2 = Trend broken, losses compound quickly
- Don't wait for bounce - exit fast
- Limit: Friday close - 1% (aggressive exit)

**10:00-10:30am: BUY P1 Positions (PATIENT)**
- Wait for market to settle after opening volatility
- Avoid wide spreads and overnight gap reactions
- Limit: Friday close + 1.5% (balanced entry)
- Better to miss entry than chase a 3%+ gap

### 📊 LIMIT ORDER STRATEGY

**SELL Limits (Aggressive):**
- Set at Friday close × 0.99 (1% below)
- Example: $100 Friday → Limit $99
- Priority: Get out fast, price less important
- If gaps down to $95, you exit at $95 (good!)
- If opens at $100.50, you exit at $100.50 (fine!)

**BUY Limits (Balanced):**
- Set at Friday close × 1.015 (1.5% above)
- Example: $95.50 Friday → Limit $96.93
- Gives room for normal overnight gaps
- Protects against chasing 3%+ moves
- ~90% fill rate vs 60% at exact Friday close

### 🚫 MID-WEEK TRADING RULES

**If BUY limit doesn't fill Monday:**
- ❌ **DON'T chase Tuesday-Thursday**
- ❌ **DON'T try to "catch the pullback"**
- ✅ **WAIT for next Friday's scan**
- ✅ **Enter different position that fills**

**Why wait?**
- Signals based on FRIDAY weekly close only
- Mid-week entry deviates from backtested strategy
- State might change by next Friday anyway
- 14+ trades/year means missing one is fine

**If SELL signal on Friday:**
- ✅ **MUST execute Monday 9:30am**
- ⚠️ If somehow missed, sell Tuesday morning
- ❌ **NEVER wait until next Friday**
- N2 detection already happened - act immediately

### 🎯 ASYMMETRIC STRATEGY

**Selling = Speed**
- Early execution (9:30am)
- Aggressive limits (-1%)
- No patience needed
- Preserve capital fast

**Buying = Patience**
- Delayed execution (10:00am)
- Flexible limits (+1.5%)
- Skip if too expensive
- Wait for next opportunity

### 💡 KEY PRINCIPLE

**Weekly discipline > Perfect execution**

The +514% backtest assumes reasonable execution at WEEKLY signals. Missing a Monday entry and waiting for next Friday is better than mid-week improvisation.

**In a hurry to stop losses, patient to enter winners.**

## 7. Export Results

Save results to CSV for record-keeping and further analysis

In [13]:
"""
GHB Portfolio Scanner - Weekly PDF Report
Generates professional PDF with market sentiment, portfolio tracking, and action items
Includes variable position sizing and risk-adjusted entry strategies
"""

import pandas as pd
from reportlab.lib.pagesizes import letter, A4
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.lib import colors
from reportlab.lib.enums import TA_LEFT, TA_CENTER
from datetime import datetime
from pathlib import Path

# Create PDF filename with timestamp
datetime_str = datetime.now().strftime('%Y%m%d_%H%M')
results_dir = Path('../ghb_scanner_results')
results_dir.mkdir(exist_ok=True)

# Archive old PDFs
archive_dir = results_dir / 'archive'
archive_dir.mkdir(exist_ok=True)
for old_file in results_dir.glob('ghb_weekly_report_*.pdf'):
    target = archive_dir / old_file.name
    if target.exists():
        target.unlink()
    old_file.rename(target)

pdf_file = results_dir / f'ghb_weekly_report_{datetime_str}.pdf'
doc = SimpleDocTemplate(str(pdf_file), pagesize=letter)
story = []
styles = getSampleStyleSheet()

# Custom styles
title_style = ParagraphStyle('CustomTitle', parent=styles['Heading1'], fontSize=16, textColor=colors.HexColor('#1f4788'), spaceAfter=12)
summary_style = ParagraphStyle('Summary', parent=styles['Heading2'], fontSize=12, textColor=colors.HexColor('#1f4788'), spaceAfter=6)
subtitle_style = ParagraphStyle('Subtitle', parent=styles['Normal'], fontSize=10, textColor=colors.grey, spaceAfter=12)

# Helper functions for variable position sizing
def get_position_allocation(ticker):
    # If ticker has custom allocation, return it
    if ticker in position_allocations:
        return position_allocations[ticker]
    
    # Calculate remaining allocation for non-custom stocks
    total_custom_alloc = sum(position_allocations.values())
    remaining_alloc = 100 - total_custom_alloc
    
    # Split remaining allocation among (max_positions - custom_count) stocks
    num_non_custom_slots = max_positions - len(position_allocations)
    if num_non_custom_slots > 0:
        return remaining_alloc / num_non_custom_slots
    else:
        return position_size_pct  # Fallback if all positions are custom

def get_position_value(ticker):
    allocation_pct = get_position_allocation(ticker)
    return starting_cash * allocation_pct / 100

# Title Page
story.append(Paragraph("GHB Weekly Portfolio Scanner Report", title_style))
story.append(Paragraph(f"Generated: {datetime.now().strftime('%A, %B %d, %Y at %I:%M %p')}", subtitle_style))
story.append(Spacer(1, 0.2*inch))

# Quick Summary
pct_bullish = (len(p1_signals) / total_universe * 100) if total_universe > 0 else 0
if pct_bullish > 40:
    sentiment = "BULLISH"
elif pct_bullish > 20:
    sentiment = "NEUTRAL"
else:
    sentiment = "BEARISH"

summary_line = f"<b>Week {strategy_week}:</b> {len(p1_signals)} Buy Opportunities | {len(n2_signals)} Sell Signals | Market: {sentiment}"
story.append(Paragraph(summary_line, styles['Normal']))
story.append(Spacer(1, 0.2*inch))

# Entry Risk Analysis with colored backgrounds
if len(p1_signals) > 0:
    story.append(Paragraph("Entry Risk Analysis", summary_style))
    risk_data = [['Stock', 'Current', 'Support', 'Distance', 'Risk', 'Strategy']]
    risk_colors = []
    
    for _, row in p1_signals.head(min(10, len(p1_signals))).iterrows():
        ticker = row['Ticker']
        current = row['Close']
        support = row['Support']
        distance = row['To_Support_%']
        
        if distance < 3:
            risk_icon = 'LOW'
            strategy = '100%'
            risk_color = colors.HexColor('#d4edda')
        elif distance < 5:
            risk_icon = 'LOW-MOD'
            strategy = '75%'
            risk_color = colors.HexColor('#d4edda')
        elif distance < 10:
            risk_icon = 'MOD'
            strategy = '50%'
            risk_color = colors.HexColor('#fff3cd')
        elif distance < 15:
            risk_icon = 'HIGH'
            strategy = '30%'
            risk_color = colors.HexColor('#f8d7da')
        else:
            risk_icon = 'WAIT'
            strategy = 'WAIT'
            risk_color = colors.HexColor('#e9ecef')
        
        risk_data.append([ticker, f'${current:.2f}', f'${support:.2f}', f'{distance:+.1f}%', risk_icon, strategy])
        risk_colors.append(risk_color)
    
    risk_table = Table(risk_data, colWidths=[0.6*inch, 0.9*inch, 0.9*inch, 0.8*inch, 0.9*inch, 0.9*inch])
    
    table_style = [
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#1f4788')),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 8),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 6),
        ('BACKGROUND', (0, 1), (-1, -1), colors.HexColor('#f8f9fa')),
        ('GRID', (0, 0), (-1, -1), 0.5, colors.grey),
        ('FONTSIZE', (0, 1), (-1, -1), 7),
        ('TOPPADDING', (0, 1), (-1, -1), 3),
        ('BOTTOMPADDING', (0, 1), (-1, -1), 3),
    ]
    
    for i, color in enumerate(risk_colors, start=1):
        table_style.append(('BACKGROUND', (4, i), (4, i), color))
        table_style.append(('FONTNAME', (4, i), (4, i), 'Helvetica-Bold'))
    
    risk_table.setStyle(TableStyle(table_style))
    story.append(risk_table)
    story.append(Spacer(1, 0.2*inch))

# Re-optimization alerts
if len(reopt_alerts) > 0:
    story.append(Paragraph("Universe Health Alerts", summary_style))
    for alert in reopt_alerts:
        story.append(Paragraph(f"• {alert[0]}: {alert[1]} - {alert[2]}", styles['Normal']))
    story.append(Spacer(1, 0.2*inch))

# Action Items for Monday
story.append(Paragraph("Action Items for Monday", summary_style))
action_items = []

# Point 1: SELL
if len(df_positions_enriched) > 0 and 'Signal' in df_positions_enriched.columns:
    n2_positions = df_positions_enriched[df_positions_enriched['Signal'] == 'N2']
else:
    n2_positions = pd.DataFrame()

if len(n2_positions) > 0:
    total_sell_value = n2_positions['Position_Value'].sum()
    action_items.append(f"1. SELL ALL {len(n2_positions)} N2 position(s) - ${total_sell_value:,.0f}")
    for _, pos in n2_positions.iterrows():
        ticker = pos['Ticker']
        shares = pos['Shares']
        pl_pct = pos['P/L_%']
        pl_sign = "+" if pl_pct >= 0 else ""
        action_items.append(f"   {ticker}: SELL {shares} shares ({pl_sign}{pl_pct:.1f}%)")
else:
    if len(df_positions_enriched) > 0:
        action_items.append("1. No positions to sell - all holdings healthy")
    else:
        action_items.append("1. No current holdings - portfolio is 100% cash")

# Point 2: BUY
if len(p1_signals) > 0:
    # Filter for acceptable risk: only buy stocks within 10% of support (LOW/LOW-MOD/MOD risk)
    safe_entries = p1_signals[p1_signals['To_Support_%'] < 10].copy()
    
    current_positions = len(df_positions_enriched)
    positions_to_add = min(max_positions - current_positions, len(safe_entries))
    
    top_buys = safe_entries.head(positions_to_add)
    conservative_deploy = 0
    buy_details = []
    
    for _, row in top_buys.iterrows():
        ticker = row['Ticker']
        friday_close = row['Close']
        distance = row['To_Support_%']
        ticker_position_value = get_position_value(ticker)
        ticker_allocation_pct = get_position_allocation(ticker)
        
        if distance < 3:
            pilot_pct = 100
        elif distance < 5:
            pilot_pct = 75
        elif distance < 10:
            pilot_pct = 50
        elif distance < 15:
            pilot_pct = 30
        else:
            pilot_pct = 0
        
        pilot_value = ticker_position_value * pilot_pct / 100
        conservative_deploy += pilot_value
        
        if pilot_pct > 0:
            shares = int(pilot_value / friday_close)
            limit_price = friday_close * 1.015
            actual_cost = shares * limit_price
            alloc_note = f" [{ticker_allocation_pct}%]" if ticker_allocation_pct != position_size_pct else ""
            
            buy_details.append({
                'ticker': ticker,
                'alloc_note': alloc_note,
                'shares': shares,
                'limit_price': limit_price,
                'actual_cost': actual_cost,
                'pilot_pct': pilot_pct
            })
    
    if conservative_deploy > 0:
        action_items.append(f"2. BUY {len(buy_details)} positions - Deploy ${conservative_deploy:,.0f}")
        for detail in buy_details:
            action_items.append(f"   {detail['ticker']}{detail['alloc_note']}: {detail['shares']} shares @ ${detail['limit_price']:.2f} = ${detail['actual_cost']:,.0f} ({detail['pilot_pct']}%)")
    else:
        action_items.append("2. BUY signals available but ALL extended >10% - WAIT for pullbacks")
else:
    action_items.append("2. No new buys available - hold cash")

# Point 3: MONITOR
if len(df_positions_enriched) > 0:
    owned_tickers = df_positions_enriched['Ticker'].tolist()
    hold_owned = hold_signals[hold_signals['Ticker'].isin(owned_tickers)]
    if len(hold_owned) > 0:
        action_items.append(f"3. Monitor {len(hold_owned)} positions: {', '.join(hold_owned['Ticker'].tolist())}")
    else:
        action_items.append(f"3. All {len(df_positions_enriched)} positions healthy (P1)")

for item in action_items:
    story.append(Paragraph(item, styles['Normal']))
    story.append(Spacer(1, 0.05*inch))

# Portfolio Holdings
story.append(Spacer(1, 0.2*inch))
story.append(Paragraph("Current Portfolio Holdings", summary_style))
story.append(Spacer(1, 0.1*inch))

if len(df_positions_enriched) > 0:
    total_value = df_positions_enriched['Position_Value'].sum()
    total_pl = df_positions_enriched['P/L_$'].sum()
    total_pl_pct = (total_pl / (total_value - total_pl) * 100) if (total_value - total_pl) > 0 else 0
    cash_remaining = starting_cash - total_value
    deployed_pct = int(total_value / starting_cash * 100)
    
    position_summaries = [
        f"<b>Total Value:</b> ${total_value:,.0f} ({deployed_pct}% deployed)",
        f"<b>Cash Remaining:</b> ${cash_remaining:,.0f}",
        f"<b>Total P/L:</b> ${total_pl:,.0f} ({total_pl_pct:+.1f}%)",
        f"<b>Positions:</b> {len(df_positions)}/{max_positions}"
    ]
    
    for summary in position_summaries:
        story.append(Paragraph(summary, styles['Normal']))
    
    story.append(Spacer(1, 0.1*inch))
    
    holdings_data = [['Ticker', 'Shares', 'Entry', 'Current', 'P/L %', 'State']]
    for _, pos in df_positions.iterrows():
        holdings_data.append([
            pos['Ticker'],
            str(pos['Shares']),
            f"${pos['Entry_Price']:.2f}",
            f"${pos['Current_Price']:.2f}",
            f"{pos['P/L_%']:+.1f}%",
            pos['Signal']
        ])
    
    holdings_table = Table(holdings_data, colWidths=[0.8*inch, 0.8*inch, 0.8*inch, 0.8*inch, 0.8*inch, 0.8*inch])
    holdings_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#1f4788')),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 8),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 6),
        ('BACKGROUND', (0, 1), (-1, -1), colors.HexColor('#f0f8ff')),
        ('GRID', (0, 0), (-1, -1), 0.5, colors.grey),
        ('FONTSIZE', (0, 1), (-1, -1), 8),
        ('TOPPADDING', (0, 1), (-1, -1), 4),
        ('BOTTOMPADDING', (0, 1), (-1, -1), 4),
    ]))
    story.append(holdings_table)
    
    if len(n2_positions) > 0:
        story.append(Spacer(1, 0.1*inch))
        alert_style = ParagraphStyle('Alert', parent=styles['Normal'], textColor=colors.red, fontSize=10)
        story.append(Paragraph(f"<b>WARNING:</b> {len(n2_positions)} position(s) in N2 - SELL Monday!", alert_style))

# Re-Optimization Alerts
if len(reopt_alerts) > 0:
    story.append(Spacer(1, 0.2*inch))
    alert_header_style = ParagraphStyle('AlertHeader', parent=styles['Heading3'], fontSize=12, 
                                       textColor=colors.red if alert_severity == 'RED' else colors.orange, spaceAfter=6)
    story.append(Paragraph(f"Universe Health Check: {alert_severity}", alert_header_style))
    
    alert_data = [['Condition', 'Status', 'Action Required']]
    for alert in reopt_alerts:
        alert_data.append(alert)
    
    alert_table = Table(alert_data, colWidths=[1.8*inch, 2.0*inch, 2.2*inch])
    alert_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#1f4788')),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 9),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 8),
        ('BACKGROUND', (0, 1), (-1, -1), colors.HexColor('#fff3cd') if alert_severity == 'YELLOW' else colors.HexColor('#f8d7da')),
        ('GRID', (0, 0), (-1, -1), 0.5, colors.grey),
        ('FONTSIZE', (0, 1), (-1, -1), 8),
        ('TOPPADDING', (0, 1), (-1, -1), 4),
        ('BOTTOMPADDING', (0, 1), (-1, -1), 4),
        ('VALIGN', (0, 0), (-1, -1), 'TOP'),
    ]))
    story.append(alert_table)
    story.append(Spacer(1, 0.1*inch))
    
    if alert_severity == 'RED':
        story.append(Paragraph("<b>CRITICAL:</b> Re-screen S&P 500 before next week", styles['Normal']))
    elif alert_severity == 'YELLOW':
        story.append(Paragraph("<b>RECOMMENDED:</b> Plan re-optimization within 1-2 months", styles['Normal']))

# Optional signal reference page
if len(p1_signals) + len(hold_signals) + len(n2_signals) > 0:
    story.append(PageBreak())
    story.append(Paragraph("Quick Reference: All Signals", summary_style))
    story.append(Spacer(1, 0.1*inch))

if len(p1_signals) > 0:
    story.append(Paragraph(f"P1 BUY Signals ({len(p1_signals)})", ParagraphStyle('Subheading', parent=styles['Normal'], fontSize=10, textColor=colors.darkgreen, spaceAfter=6, alignment=TA_LEFT)))
    p1_data = [['Ticker', 'Price', 'Support', 'Distance', 'ROC%']]
    for _, row in p1_signals.iterrows():
        p1_data.append([row['Ticker'], f"${row['Close']:.2f}", f"${row['Support']:.2f}", f"{row['To_Support_%']:+.1f}%", f"{row['ROC_4W_%']:.1f}%"])
    
    p1_table = Table(p1_data, colWidths=[1.0*inch, 1.0*inch, 1.0*inch, 1.0*inch, 1.0*inch])
    p1_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.darkgreen),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 8),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 6),
        ('BACKGROUND', (0, 1), (-1, -1), colors.HexColor('#d4edda')),
        ('GRID', (0, 0), (-1, -1), 0.5, colors.grey),
        ('FONTSIZE', (0, 1), (-1, -1), 8),
        ('TOPPADDING', (0, 1), (-1, -1), 4),
        ('BOTTOMPADDING', (0, 1), (-1, -1), 4),
    ]))
    story.append(p1_table)
    story.append(Spacer(1, 0.15*inch))

if len(hold_signals) > 0:
    story.append(Paragraph(f"HOLD Signals ({len(hold_signals)})", ParagraphStyle('Subheading', parent=styles['Normal'], fontSize=10, textColor=colors.orange, spaceAfter=6, alignment=TA_LEFT)))
    hold_data = [['Ticker', 'Price', 'Distance']]
    for _, row in hold_signals.iterrows():
        hold_data.append([row['Ticker'], f"${row['Close']:.2f}", f"{row['Distance_%']:+.1f}%"])
    
    hold_table = Table(hold_data, colWidths=[1.5*inch, 1.5*inch, 1.0*inch])
    hold_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.orange),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 8),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 6),
        ('BACKGROUND', (0, 1), (-1, -1), colors.HexColor('#fff3cd')),
        ('GRID', (0, 0), (-1, -1), 0.5, colors.grey),
        ('FONTSIZE', (0, 1), (-1, -1), 8),
        ('TOPPADDING', (0, 1), (-1, -1), 4),
        ('BOTTOMPADDING', (0, 1), (-1, -1), 4),
    ]))
    story.append(hold_table)
    story.append(Spacer(1, 0.15*inch))

if len(n2_signals) > 0:
    story.append(Paragraph(f"N2 SELL Signals ({len(n2_signals)})", ParagraphStyle('Subheading', parent=styles['Normal'], fontSize=10, textColor=colors.red, spaceAfter=6, alignment=TA_LEFT)))
    sell_data = [['Ticker', 'Price', 'Below SMA']]
    for _, row in n2_signals.iterrows():
        sell_data.append([row['Ticker'], f"${row['Close']:.2f}", f"{row['Distance_%']:.1f}%"])
    
    sell_table = Table(sell_data, colWidths=[1.5*inch, 1.5*inch, 1.5*inch])
    sell_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.red),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 8),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 6),
        ('BACKGROUND', (0, 1), (-1, -1), colors.HexColor('#f8d7da')),
        ('GRID', (0, 0), (-1, -1), 0.5, colors.grey),
        ('FONTSIZE', (0, 1), (-1, -1), 8),
        ('TOPPADDING', (0, 1), (-1, -1), 4),
        ('BOTTOMPADDING', (0, 1), (-1, -1), 4),
    ]))
    story.append(sell_table)

# Build PDF
doc.build(story)
print(f"✅ Simplified PDF report generated: {pdf_file}")
print(f"   📋 Includes: Action items, Portfolio holdings, Entry risk table (color-coded), Signal reference")

✅ Simplified PDF report generated: ..\ghb_scanner_results\ghb_weekly_report_20260116_0818.pdf
   📋 Includes: Action items, Portfolio holdings, Entry risk table (color-coded), Signal reference


## 8. Export Data

Save CSV file to the same location as PDF reports

In [14]:
# Export results to CSV in same location as PDF
from pathlib import Path
from datetime import datetime

# Use same directory as PDF reports
results_dir = Path('../ghb_scanner_results')
results_dir.mkdir(exist_ok=True)

# Archive old CSV files
archive_dir = results_dir / 'archive'
archive_dir.mkdir(exist_ok=True)
for old_file in results_dir.glob('ghb_strategy_signals_*.csv'):
    target = archive_dir / old_file.name
    if target.exists():
        target.unlink()  # Delete existing file in archive
    old_file.rename(target)

# Create new CSV file
datetime_str = datetime.now().strftime('%Y%m%d_%H%M')
csv_file = results_dir / f'ghb_strategy_signals_{datetime_str}.csv'
df_results.to_csv(csv_file, index=False)

print(f"\n✅ CSV exported: {csv_file}")
print(f"   Rows: {len(df_results)} stocks")
print(f"   Location: {results_dir.absolute()}")


✅ CSV exported: ..\ghb_scanner_results\ghb_strategy_signals_20260116_0818.csv
   Rows: 12 stocks
   Location: c:\workspace\portfolio_analyser\notebooks\..\ghb_scanner_results


## GHB Strategy Quick Reference

### Your AI-Focused Portfolio
**11 stocks:** AI/Tech leaders aligned with AI dominance thesis (2023-2032)  
**Expected:** 56.51% CAGR, ~14 trades/year, 40% win rate (2022-2025 backtest)  
**Strategy:** Concentrated bet on AI sector boom vs diversified S&P 500 approach

### Entry Rules (BUY)
- State = P1 (Gold)
- Price > 200-day SMA
- Strong momentum (ROC > 5% OR distance > 10%)
- **NEW: Smart Entry Quality (RSI-based)**
  - 🔥 **PULLBACK BUY** (Priority #1): P1 + negative ROC = buy the dip
  - ✅ **HEALTHY BUY** (Priority #2): RSI <70, Distance <30% = ideal entry
  - ⚠️ **EXTENDED** (Caution): RSI 70-80 or Distance 30-40% = enter small
  - 🚨 **OVERHEATED** (Avoid): RSI >80 + Distance >40% = wait for pullback

### Hold Rules
- **P1 (Gold):** Continue holding, consider adding
- **P2 (Gray):** Hold through consolidation
- **N1 (Gray):** Hold through shallow pullback

### Exit Rules (SELL)
- State = N2 (Blue)
- Price < 200-day SMA
- Weak momentum
- **Execute Monday at open!**

### Expected Performance
- **CAGR:** 56.51% (AI/Tech focused, 2022-2025 backtest)
- **Total Return (3.3yr):** 332.54% ($110k → $476k)
- **Trades Per Year:** ~14 (more active than diversified approach)
- **Win Rate:** 40% (lower but bigger wins)
- **Best Trade:** NVDA +516% (777 days)
- **Avg Winner:** +135% (NU, NVDA, TSM, GOOG)
- **Hold Period:** 6-24+ months for winners
- **Max Drawdown:** Not yet measured (limited history)

### Risk Management
- Max 10% per position ($11,000 per trade)
- Up to 10 concurrent positions
- **HIGH CONCENTRATION RISK:** All tech/AI exposure
- **Thesis-Dependent:** Requires AI dominance 2023-2032
- **Overextension Risk:** Use RSI warnings to avoid buying tops
- Weekly monitoring only (10-15 minutes)

### Execution Guidelines
**Monday 9:30-10:00am: SELL N2 (URGENT)**
- Execute all sells first 30 minutes
- Limit: Friday close - 1% (aggressive exit)
- Don't wait for bounce - exit fast
- If missed Monday, sell Tuesday morning

**Monday 10:00-10:30am: BUY P1 (PATIENT)**
- Wait for market to settle after open
- Limit: Friday close + 1.5% (balanced entry)
- If limit doesn't fill, WAIT for next Friday
- Don't chase mid-week (maintain weekly discipline)

**Key Principle:** In a hurry to stop losses, patient to enter winners

---
**Next Steps:**
1. Review signals above
2. Execute SELLS first (9:30-10:00am Monday)
3. Execute BUYS second (10:00-10:30am Monday)

4. Update portfolio CSV Monday evening**Custom Universe:** See `backtest/data/custom_tech_stocks.txt` for AI/tech stock list  **Analysis:** See `backtest/analyze_custom.py` for complete backtest breakdown

5. Run this notebook again next Friday

**Documentation:** See `backtest/BACKTEST_BIAS_ANALYSIS.md` for unbiased methodology  